# Llama-3.1 function calling

In [3]:
from dotenv import load_dotenv
import os
import openai
import requests
import json

In [4]:
load_dotenv()
api_key = os.getenv("NAVIGATOR_API_KEY")
api_base_url = os.getenv("NAVIGATOR_API_BASE_URL")
canvas_api_token = os.getenv("CANVAS_API_TOKEN")

model = "llama-3.1-70b-instruct"

In [ ]:
headers = {'Authorization': f'Bearer {canvas_api_token}'}
url = 'https://ufldev.test.instructure.com/api/v1/courses/'
course_id = '180/'

r = requests.get(url, headers = headers)

In [ ]:
course_details_url = f"{url}/{course_id}"
resp_course = requests.get(course_details_url, headers=headers)

In [ ]:
resp_course.json()

In [ ]:
client = openai.OpenAI(
    api_key=api_key,
    base_url=api_base_url # LiteLLM Proxy is OpenAI compatible, Read More: https://docs.litellm.ai/docs/proxy/user_keys
)

In [ ]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City and country e.g. Bogotá, Colombia"
                }
            },
            "required": [
                "location"
            ],
        },
    }
}]

In [ ]:
from typing import List

def llm_api_call(client, model: str, messages: List[str], max_tokens: int = 100) -> str:
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools
    )
    return response.choices[0].message.content

In [ ]:
message = [{
    'role': 'user',
    'content': 'What\'s the weather in New York?'
}]
response = llm_api_call(client, model, message)

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage
from langchain.tools import tool
from langchain.agents import create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
@tool
def get_current_weather(location: str) -> str:
    """Returns weather data for a location."""
    return f"Weather in {location}: Sunny, 22°C"

chat = ChatOpenAI(
    openai_api_base=api_base_url,
    model=model,
    api_key=api_key
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),  # Required for agent's intermediate steps
])

agent = create_tool_calling_agent(chat, [get_current_weather], prompt)

In [ ]:
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.core.agent import FunctionCallingAgent
from llama_index.core.tools import FunctionTool


llm = AzureOpenAI(
    engine=model,               # model_name on litellm proxy
    temperature=0.0,
    azure_endpoint=api_base_url, # litellm proxy endpoint
    api_key=api_key,                    # litellm proxy API Key
    api_version="2023-07-01-preview"
)

def add(x: int, y: int) -> int:
    """Add two numbers."""
    return x + y

add_tool = FunctionTool.from_defaults(add)

# Create agent with explicit function support
agent = FunctionCallingAgent.from_tools(
    tools=[add_tool],
    llm=llm,
    verbose=True,
    allow_parallel_tool_calls=False  # Disable if unsupported[12]
)

response = agent.chat("What is 7 + 3?")


> Running step 20cff41c-22fe-4b99-a8a4-8c7235f4a17e. Step input: What is 7 + 3?
Added user message to memory: What is 7 + 3?


NotFoundError: Error code: 404 - {'detail': 'Not Found'}

> Running step 3f2480eb-cc13-4742-a8a3-f795dd91d809. Step input: What is 7 + 3?
Added user message to memory: What is 7 + 3?


NotFoundError: Error code: 404 - {'detail': 'Not Found'}